In [1]:
from selenium.webdriver.common.keys import Keys                         
from selenium.webdriver.support.ui import Select                
from selenium.webdriver.support.ui import WebDriverWait                    
from selenium.webdriver.support import expected_conditions as EC           
from selenium.webdriver.common.by import By                                
from selenium import webdriver                      
import time                                         
from bs4 import BeautifulSoup                                               
import matplotlib.pyplot as plt                             

## login to Instagram using username and password

In [20]:
driver = webdriver.Chrome(executable_path = '/Users/91771/Downloads/chromedriver/chromedriver')
driver.get('https://www.instagram.com/')                                        
wait = WebDriverWait(driver, 10)                                            
time.sleep(5)                                                                   
username = driver.find_element_by_name('username')
username.send_keys('SAMPLE USERNAME')
password = driver.find_element_by_name('password')
password.send_keys('SAMPLE PASSWORD')
password.submit()                                                       
exit_btn = wait.until(EC.presence_of_element_located((By.XPATH,'//div/button[contains(@class,"HoLwm")]')))  
exit_btn.click()                                                                

## 1.1Top 5 Food Handles with maximum number of followers

In [7]:
a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]')))  
a.send_keys('food')                                                                            
time.sleep(3)
handles = driver.find_elements_by_xpath('//div[@class = "fuqBx"]/a["href"]')                
food_list = []
for i in handles:
    if 'explore' in i.get_attribute('href'):
        continue
    else:
        s = i.get_attribute('href').split('/')
        if s[3]!="food" and s[3]!='foodytops': 
            food_list.append(s[3])             
a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]'))) 
a.clear()
a = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'coreSpriteSearchClear')))       
a.click()
new_food_list = food_list[:10]                                                                
d={}
for ele in new_food_list:
    a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]')))        
    a.send_keys(ele)
    first_search = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'z556c')))                 
    first_search.click()
    time.sleep(2)
    followers_button = wait.until(EC.presence_of_element_located((By.XPATH,'//a[contains(@class,"-nal3")]')))
    followers_count = driver.find_elements_by_xpath('//a[contains(@class,"-nal3")]/span')                      
    follow_count = followers_count[0].get_attribute('title')                                                
    p=follow_count.split(",")                                       
    ccount=int("".join(p))                                                
    d[ele]=ccount                                                         
    a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]'))) 
    a.clear()
c=1
top_5_blog=[]
max_follow=[]


for i in sorted (d.items(), key = lambda kv: kv[1],reverse=True):
    top_5_blog.append(i[0],)                                
    max_follow.append(i[1])                                      
    print(i)               
    c+=1
    if c>5:
        break

('yourfoodlab', 909168)
('foodrush.recipe', 660411)
('dilsefoodie', 369830)
('foodtalkindia', 292665)
('pune_food_blogger', 51181)


### 1.2 Now Find the number of posts these handles have done in the previous 3 days. 

In [16]:
no_of_post={}
for handle in top_5_blog:
    search = driver.find_element_by_xpath('//div[contains(@class,"LWmhU")]/input')   
    search.send_keys(handle)                                                         
    b = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'z556c'))) 
    first_search = driver.find_element_by_class_name('z556c')          
    first_search.click()
    a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]')))  
    a.clear()
    count = 0
    i=0
    for i in range(7):
        driver.execute_script('window.scrollBy(0,document.body.scrollHeight);')                
        time.sleep(1)
    data=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]')                  
    for j in range(7):  
        driver.execute_script('window.scrollBy(0,-document.body.scrollHeight);')               
        time.sleep(1)
    driver.execute_script('window.scrollBy(0,500);')                                           
    final=data
    for i in range(len(final)):
        post=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]')              
        driver.execute_script('window.scrollBy(0,80);')                                      
        wait = WebDriverWait(driver, 10)                                                    
        post[i].click()                                                             
        test= wait.until(EC.presence_of_element_located((By.XPATH,'//time[contains(@class,"FH9sR")]'))) 
        val=test.text                            
        if val[-1]=='d':                         
            new_val=int(val[0:-1])
            if new_val<4:                        
                count=count+1
            else:
                new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')
                new.click()
                break
        else:
            if val[-1]=='h':                     
                count=count+1
            elif val[-1]=='m':                   
                count=count+1
            else:
                new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')
                new.click()
                break
        time.sleep(5)        
        close_btn = driver.find_element_by_class_name('ckWGn')
        time.sleep(2)
        close_btn.click()                                       
        time.sleep(1)
    no_of_post[handle]=count                                
handles=[]
number_posts=[]
for i in no_of_post:
    handles.append(i)                                            
    number_posts.append(no_of_post[i])
for i in range(len(handles)):
    print(handles[i],'',number_posts[i])

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ckWGn"}
  (Session info: chrome=80.0.3987.149)


## 2 Your friend also needs a list of hashtags that he should use in his posts.
## 2.1 Open the 5 handles you obtained in the last question, and scrape the content of the first 10 posts of each handle.

In [18]:
top_5_handles={}
for handle in top_5_blog:
    top_10_post={}
    search = driver.find_element_by_xpath('//div[contains(@class,"LWmhU")]/input')          
    search.send_keys(handle)
    b = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'z556c')))                 
    first_search = driver.find_element_by_class_name('z556c')
    first_search.click()
    a= wait.until(EC.presence_of_element_located((By.XPATH,'//input[contains(@class,"XTCLo")]')))    
    a.clear()
    for i in range(1):
        driver.execute_script('window.scrollBy(0,document.body.scrollHeight);')     
        time.sleep(1)
    data=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]')                   
    for j in range(1):
        driver.execute_script('window.scrollBy(0,-document.body.scrollHeight);')                    
        time.sleep(1)
    driver.execute_script('window.scrollBy(0,500);')                                                
    for i in range(10):
        post=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]')                  
        driver.execute_script('window.scrollBy(0,80);')                                         
        driver.implicitly_wait(10) 
        wait = WebDriverWait(driver, 10)
        post[i].click()                                                                         
        data=driver.find_element_by_xpath('//div[contains(@class,"C4VMK")]/span')                   
        value=data.text                                                                             
        top_10_post["post"+str(i+1)]=value    
        new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')                  
        new.click()
    top_5_handles[handle]=top_10_post
for key in top_5_handles.keys():
    p=top_5_handles[key]
    print(key)
    i=0
    for key in p:
        print("**************************post",i+1,"*******************************")
        print(p[key])
        i=i+1
        print()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@class,"ckWGn")]"}
  (Session info: chrome=80.0.3987.149)


## 3.You need to also calculate average followers : likes ratio for the obtained handles Followers : Likes ratio is calculated as follows:
## 3.1 Find out the likes of the top 10 posts of the 5 handles obtained earlier.

In [21]:
from selenium.common.exceptions import NoSuchElementException               
top_5_post_like={}                                                  
for t in top_5_blog:                                                        
    post_likes={}
    count=0
    data=driver.find_element_by_xpath('//input[contains(@class,"XTCLo")]')  
    data.send_keys(t)                                                       
    driver.implicitly_wait(10)
    data=driver.find_element_by_xpath('//span[contains(@class,"Ap253")]')   
    data.click()
    driver.implicitly_wait(10)
    for i in range(12):
        driver.execute_script('window.scrollBy(0,2000);')           
        time.sleep(1)
    data=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]') 
    for j in range(12): 
        driver.execute_script('window.scrollBy(0,-2000);')          
        time.sleep(1)
    driver.execute_script('window.scrollBy(0,500);')                
    final=data
    driver.implicitly_wait(10)
    for p in range(len(final)):
        if count>9:
            break  
        try:
            data=driver.find_elements_by_xpath('//div[contains(@class,"_9AhH0")]')  
            driver.execute_script('window.scrollBy(0,80);')
            final=len(data)
            driver.implicitly_wait(10)
            data[p].click()
            driver.implicitly_wait(10)
            data=driver.find_element_by_xpath('//button[contains(@class,"sqdOP ")]//span') 
            val=data.text
            if val=='':
                new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')     
                new.click()
                continue
            if val[0]=='V':
                new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')
                new.click()
                continue
            val=val.replace(',','')
            ans=val
            count=count+1
            number='post'+str(count)
            post_likes[number]=ans
            new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')
            new.click()
            
        except NoSuchElementException:
            new=driver.find_element_by_xpath('//button[contains(@class,"ckWGn")]')
            new.click()
    
    top_5_post_like[t]=post_likes    
    
print(top_5_post_likes)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@class,"ckWGn")]"}
  (Session info: chrome=80.0.3987.149)
